# Introdução à MPI - Message Passing Interface
#### _bootcamp_ da Escola Supercomputador Santos Dumont - 2025
por Calebe de Paula Bianchini

### Você lembra como funciona MPI?

Em MPI, a aplicação será dividida em diversos processos que se comunicarão entre si por meio de primitivas de envio de mensagem. De forma geral, a figura a seguir epresenta a estrutura de um programa MPI.

![MPI](./img/mpi_estrutura_programa.png)

A partir do momento que a aplicação está em funcionamento, diversos processos poderão comunicar-se identificando-se por meio de um **comunicador**. É por meio desse comunicador que são estabelecidos os grupos e o contexto (identificador) de cada processo. Veja a figura a seguir.

![Comunicador](./img/mpi_comunicador.png)

O programa a seguir segue a estrutura definida para um programa MPI, e sua comunicação acontece por meio do comunicador padrão: MPI\_COMM\_WORLD.

In [ ]:
%%writefile mpi_simples.c

#include <stdio.h>
#include <string.h>
#include "mpi.h"

int main(int argc, char *argv[]) { /* mpi_simples.c  */
int meu_ranque, num_procs;
int origem, destino = 0, etiq = 1;
char mensagem[200];
MPI_Status estado;
    /* Chamada inicial para o MPI */
    MPI_Init(&argc, &argv);
    /* Determina o ranque e número de processos em execução */
    MPI_Comm_rank(MPI_COMM_WORLD, &meu_ranque);
    MPI_Comm_size(MPI_COMM_WORLD, &num_procs);
    /* Todos processos com ranque diferente de 0 enviam uma mensagem */
    if (meu_ranque != destino) {
        sprintf(mensagem, "Processo %d está vivo!", meu_ranque);
        MPI_Send(mensagem, strlen(mensagem)+1, MPI_CHAR, destino, etiq, MPI_COMM_WORLD);
    } else {
    /* Processo com ranque 0 recebe num_procs-1 mensagens */
        for (origem = 1; origem < num_procs; origem++) {
             MPI_Recv(mensagem, 200, MPI_CHAR, origem, etiq, MPI_COMM_WORLD, &estado);
    /* Imprime as mensagens recebidas */
             printf("%s\n",mensagem);
        }
    }
    MPI_Finalize();
    return(0);
}

Para a compilação usando MPI, é necessário fazer a preparação do ambiente. A seguinr apresentamos uma proposta para essa configuração:

In [ ]:
!apt install -yyy openmpi-bin openmpi-common libopenmpi-dev &> /dev/null
!gcc --version
!mpicc --showme:version

E, por fim, uma vez configurado o ambiente, podemos compilar o programa MPI.

In [ ]:
!mpicc mpi_simples.c -o mpi_simples -Wall -O3

Para a execução de um programa MPI é necessário indicar quantos processos serão criados. Veja o exemplo a seguir:

In [ ]:
!mpirun --allow-run-as-root --host localhost:4 -np 4 ./mpi_simples